In [1]:
!python --version

Python 3.11.12


# AHoJ-DB Užklausa

https://apoholo.cz/db/search?pdb_ids=&uniprot_ids=&ligands=ADP%20ATP%20AMP%20GTP%20GDP%20FAD%20NAD%20NAP%20NDP%20HEM%20HEC%20PLM%20OLA%20MYR%20CIT%20CLA%20CHL%20BCL%20BCB&res_threshold=1.5

Rezoliucija: 1.5 Angstromų.
Ligandai: ADP ATP AMP GTP GDP FAD NAD NAP NDP HEM HEC PLM OLA MYR CIT CLA CHL BCL BCB

# Duomenų filtravimas

In [2]:
import pandas as pd
import re

df = pd.read_csv('ahojdb_search_result.csv')
filtered_df = df[df[' num_apo_sites'] > 0]
filtered_df = filtered_df[filtered_df[' num_holo_sites'] > 0]
for index, row in filtered_df.iterrows():
    filtered_df.replace(row[' target_uniprot_ids'], re.sub('\\s.*', '', row[' target_uniprot_ids'].strip()), inplace=True)
filtered_df = filtered_df.drop_duplicates(subset=[' target_uniprot_ids'])
filtered_df.to_csv('filtered_ahojdb_search_result.csv')

# Paruošiame Apo and Holo porų duomenis

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

records = []

def parse_row(tr, prefix):
    cols = tr.find_all('td')
    d = {
        f'{prefix}_structure': cols[0].get_text(strip=True),
        f'{prefix}_aligned_chain': cols[1].get_text(strip=True),
        f'{prefix}_resolution': cols[2].get_text(strip=True),
        f'{prefix}_MBR': cols[3].get_text(strip=True),
        f'{prefix}_disorder_percent': cols[4].get_text(strip=True),
        f'{prefix}_RMSD': float(cols[5].get_text(strip=True)),
        f'{prefix}_pocket_RMSD': cols[6].get_text(strip=True),
        f'{prefix}_TM_score': cols[7].get_text(strip=True),
        f'{prefix}_SASA': cols[8].get_text(strip=True),
        f'{prefix}_volume': cols[9].get_text(strip=True),
    }
    if prefix == 'holo':
        d[f'{prefix}_ligands'] = cols[10].get_text(strip=True)
    return d

for entry_key in filtered_df['entry_key']:
    url = f'http://apoholo.cz/db/entry/{entry_key}'
    html = requests.get(url).content.decode()
    soup = BeautifulSoup(html, 'html.parser')

    # --- APO table ---
    apo_table = soup.find('span', class_='label', string='Found APO Sites:') \
                    .find_next('table')
    apo_rows = apo_table.find('tbody').find_all('tr')
    apo_records = [parse_row(r, 'apo') for r in apo_rows]
    apo_rmsd_vals = [r['apo_RMSD'] for r in apo_records]

    # --- HOLO table ---
    holo_table = soup.find('span', class_='label', string='Found HOLO Sites:') \
                     .find_next('table')
    holo_rows = holo_table.find('tbody').find_all('tr')
    holo_records = [parse_row(r, 'holo') for r in holo_rows]
    holo_rmsd_vals = [r['holo_RMSD'] for r in holo_records]

    # --- Compute per‐entry stats ---
    stats = {
        'entry_key': entry_key,
        'apo_count': len(apo_rmsd_vals),
        'apo_RMSD_min': min(apo_rmsd_vals),
        'apo_RMSD_max': max(apo_rmsd_vals),
        'apo_RMSD_avg': sum(apo_rmsd_vals) / len(apo_rmsd_vals),
        'holo_count': len(holo_rmsd_vals),
        'holo_RMSD_min': min(holo_rmsd_vals),
        'holo_RMSD_max': max(holo_rmsd_vals),
        'holo_RMSD_avg': sum(holo_rmsd_vals) / len(holo_rmsd_vals),
    }

    first_apo = apo_records[0]
    first_holo = holo_records[0]
    uniprot = filtered_df.loc[
        filtered_df['entry_key'] == entry_key, ' target_uniprot_ids'
    ].iloc[0]

    rec = {
        **stats,
        **first_apo,
        **first_holo,
        'uniprot_id': uniprot
    }
    records.append(rec)

df = pd.DataFrame(records)

print(df.head())

print("=== GLOBAL APO RMSD ===")
print("  min:", df['apo_RMSD_min'].min())
print("  max:", df['apo_RMSD_max'].max())
print("  avg:", df['apo_RMSD_avg'].mean())
print("  total count:", df['apo_count'].sum())

print("=== GLOBAL HOLO RMSD ===")
print("  min:", df['holo_RMSD_min'].min())
print("  max:", df['holo_RMSD_max'].max())
print("  avg:", df['holo_RMSD_avg'].mean())
print("  total count:", df['holo_count'].sum())


         entry_key  apo_count  apo_RMSD_min  apo_RMSD_max  apo_RMSD_avg  \
0   1bab-B-HEM-147          2          0.96          1.23      1.095000   
1  1byq-A-ADP-2001         19          0.84          1.76      1.615789   
2   1d2u-A-HEM-185          1          0.86          0.86      0.860000   
3  1dgf-A-NDP-4000         11          0.23          0.38      0.302727   
4  1e6u-A-NAP-1317          1          0.46          0.46      0.460000   

   holo_count  holo_RMSD_min  holo_RMSD_max  holo_RMSD_avg apo_structure  ...  \
0         650           0.06           1.97       0.634169        5vmm/B  ...   
1         368           0.72           2.55       1.449891        5j80/A  ...   
2          48           0.19           1.06       0.689583        2ofm/X  ...   
3          30           0.18           0.49       0.326667        1dgh/D  ...   
4           6           0.10           0.45       0.280000        1gfs/A  ...   

  holo_resolution holo_MBR holo_disorder_percent holo_RMSD  ho

In [4]:
df.to_csv('apo_holo_pairs_with_RMSD_info.csv')

# Surenkame holo konformacijų struktūrų jonų ir ligandų informaciją.

In [5]:
import requests
import pandas as pd

# Function to fetch PDB information using RCSB API
def fetch_pdb_info(pdb_id):
    url = f'https://data.rcsb.org/rest/v1/core/entry/{pdb_id[:4]}'
    response = requests.get(url)

    if response.status_code != 200:
        return {
            "PDB_ID": pdb_id,
            "Status": "Failed to fetch",
            "Conformers": None,
            "Ligands": None,
            "Ions": None,
            "Other_Ligands": None,
            "Other_Ions": None
        }

    data = response.json()

    if(pdb_id == '1kmv'):
        print('data', data)

    conformers = data.get('rcsb_entry_info', {}).get('polymer_entity_count_protein', None)

    # Define known ligands and ions
    known_ligands = {"BCB", "BCL", "CHL", "CLA", "CIT", "MYR", "OLA", "PLM",
                     "HEC", "HEM", "NDP", "NAP", "NAD", "FAD", "GDP", "GTP",
                     "AMP", "ATP", "ADP"}
    known_ions = {"MG", "ZN", "CL", "CA", "NA", "MN", "K", "FE", "CU", "CO"}

    ligands = []
    ions = []
    other_ligands = []
    other_ions = []

    # Extract ligands and ions from nonpolymer_entities
    nonpolymer_entities = data.get('nonpolymer_entities', [])

    for entity in nonpolymer_entities:
        chem_comp_name = entity.get('chem_comp', {}).get('id', '')

        if chem_comp_name in known_ligands:
            ligands.append(chem_comp_name)

        elif chem_comp_name in known_ions:
            ions.append(chem_comp_name)

        else:
            chem_comp_type = entity.get('rcsb_nonpolymer_entity', {}).get('chem_comp_type', '').upper()

            if 'ION' in chem_comp_type:
                other_ions.append(chem_comp_name)

            else:
                other_ligands.append(chem_comp_name)

    bound_components = data.get('rcsb_entry_info', {}).get('nonpolymer_bound_components', [])

    for component in bound_components:
        if component not in ligands and component not in ions:

            if component in known_ligands:
                ligands.append(component)

            elif component in known_ions:
                ions.append(component)

            else:
                other_ligands.append(component)

    return {
        "PDB_ID": pdb_id,
        "Status": "Success",
        "Conformers": conformers,
        "Ligands": ligands,
        "Ions": ions,
        "Other_Ligands": other_ligands,
        "Other_Ions": other_ions
    }

df = pd.read_csv('apo_holo_pairs_with_RMSD_info.csv')
results = []

for index, row in df.iterrows():
    holo_pdb = row['holo_structure'][:4]
    apo_pdb = row['apo_structure'][:4]
    print(f"Fetching information for Holo PDB ID: {holo_pdb}")
    result = fetch_pdb_info(holo_pdb)
    result["Apo_PDB_ID"] = apo_pdb
    result["Holo_PDB_ID"] = holo_pdb
    results.append(result)

results_df = pd.DataFrame(results)

# Filter results: keep only rows where Other_Ligands are empty
filtered_results_df = results_df[(results_df['Other_Ligands'].apply(len) == 0)]

filtered_results_df = filtered_results_df[["Apo_PDB_ID", "Holo_PDB_ID", "Ligands", "Ions"]]

# Display the filtered results
print(filtered_results_df)
filtered_results_df.to_csv("apo_holo_pair_ligand_and_ion_data.csv", index=False)

Fetching information for Holo PDB ID: 2w72
Fetching information for Holo PDB ID: 6tn5
Fetching information for Holo PDB ID: 1x8q
Fetching information for Holo PDB ID: 1dgg
Fetching information for Holo PDB ID: 1e7s
Fetching information for Holo PDB ID: 3ip0
Fetching information for Holo PDB ID: 4etp
Fetching information for Holo PDB ID: 1xgk
Fetching information for Holo PDB ID: 1kmv
data {'audit_author': [{'name': 'Klon, A.E.', 'pdbx_ordinal': 1}, {'name': 'Heroux, A.', 'pdbx_ordinal': 2}, {'name': 'Ross, L.J.', 'pdbx_ordinal': 3}, {'name': 'Pathak, V.', 'pdbx_ordinal': 4}, {'name': 'Johnson, C.A.', 'pdbx_ordinal': 5}, {'name': 'Piper, J.R.', 'pdbx_ordinal': 6}, {'name': 'Borhani, D.W.', 'pdbx_ordinal': 7}], 'cell': {'angle_alpha': 90.0, 'angle_beta': 90.0, 'angle_gamma': 90.0, 'length_a': 41.3, 'length_b': 54.99, 'length_c': 82.68, 'zpdb': 4}, 'citation': [{'country': 'UK', 'id': 'primary', 'journal_abbrev': 'J.Mol.Biol.', 'journal_id_astm': 'JMOBAK', 'journal_id_csd': '0070', 'journ

In [ ]:
!rm -rf /content/cif_files

# Atsisiunčiame struktūrų CIF failus.

Peržiūrime struktūras rankiniu būdu, užtikrindami jonų egzistvimą holo struktūrose bei panašumą su apo struktūromis.




In [7]:
import os
import requests
import pandas as pd

# Function to download CIF files
def download_cif(pdb_id, output_dir):
    url = f'https://files.rcsb.org/download/{pdb_id}.cif'
    response = requests.get(url)

    if response.status_code == 200:
        filepath = os.path.join(output_dir, f"{pdb_id}.cif")
        with open(filepath, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {pdb_id}.cif")
        return True
    else:
        print(f"Failed to download: {pdb_id}.cif")
        return False

# Load the DataFrame from the previous script's filtered results
input_csv = "reviewed_apo_holo_pair_ligand_and_ion_data.tsv"
output_dir = "cif_files"
os.makedirs(output_dir, exist_ok=True)

# Read the filtered results CSV
filtered_results_df = pd.read_csv(input_csv, sep='\t')

# Download CIF files for both apo and holo IDs
for index, row in filtered_results_df.iterrows():
    apo = row['Apo_PDB_ID'][:4]
    holo = row['Holo_PDB_ID'][:4]
    print(f"Processing pair: Apo - {apo}, Holo - {holo}")
    download_cif(apo, output_dir)  # Download CIF for apo
    download_cif(holo, output_dir)  # Download CIF for holo


Processing pair: Apo - 5vmm, Holo - 2w72
Downloaded: 5vmm.cif
Downloaded: 2w72.cif
Processing pair: Apo - 5j80, Holo - 6tn5
Downloaded: 5j80.cif
Downloaded: 6tn5.cif
Processing pair: Apo - 2ofm, Holo - 1x8q
Downloaded: 2ofm.cif
Downloaded: 1x8q.cif
Processing pair: Apo - 1gfs, Holo - 1e7s
Downloaded: 1gfs.cif
Downloaded: 1e7s.cif
Processing pair: Apo - 1k6i, Holo - 1xgk
Downloaded: 1k6i.cif
Downloaded: 1xgk.cif
Processing pair: Apo - 1pdb, Holo - 1kmv
Downloaded: 1pdb.cif
Downloaded: 1kmv.cif
Processing pair: Apo - 1d6j, Holo - 1m7h
Downloaded: 1d6j.cif
Downloaded: 1m7h.cif
Processing pair: Apo - 3n80, Holo - 8dr9
Downloaded: 3n80.cif
Downloaded: 8dr9.cif
Processing pair: Apo - 2cnu, Holo - 2cnq
Downloaded: 2cnu.cif
Downloaded: 2cnq.cif
Processing pair: Apo - 1xgd, Holo - 1us0
Downloaded: 1xgd.cif
Downloaded: 1us0.cif
Processing pair: Apo - 7u8k, Holo - 4b1y
Downloaded: 7u8k.cif
Downloaded: 4b1y.cif
Processing pair: Apo - 1ufp, Holo - 5yce
Downloaded: 1ufp.cif
Downloaded: 5yce.cif
Proc

In [8]:
rmsd_df = pd.read_csv('apo_holo_pairs_with_RMSD_info.csv')
reviewed_df = pd.read_csv('reviewed_apo_holo_pair_ligand_and_ion_data.tsv', sep='\t')

reviewed_ids = set(reviewed_df['PDB_ID'].str.upper())

rmsd_df['holo_id'] = rmsd_df['holo_structure'].str[:4].str.upper()

fi_df = rmsd_df[rmsd_df['holo_id'].isin(reviewed_ids)].copy()

fi_df = fi_df[['holo_id', 'apo_count', 'apo_RMSD_min', 'apo_RMSD_max', 'apo_RMSD_avg', 'holo_count', 'holo_RMSD_min', 'holo_RMSD_max', 'holo_RMSD_avg']]
print(fi_df)

    holo_id  apo_count  apo_RMSD_min  apo_RMSD_max  apo_RMSD_avg  holo_count  \
0      2W72          2          0.96          1.23      1.095000         650   
1      6TN5         19          0.84          1.76      1.615789         368   
2      1X8Q          1          0.86          0.86      0.860000          48   
4      1E7S          1          0.46          0.46      0.460000           6   
7      1XGK          1          0.20          0.20      0.200000          28   
..      ...        ...           ...           ...           ...         ...   
146    8CRH          1          0.99          0.99      0.990000           4   
149    5J4L          7          0.86          1.31      1.094286          78   
150    8D0U          1          0.24          0.24      0.240000           6   
152    8U96          3          0.90          1.29      1.090000           4   
153    9B20          2          1.13          1.13      1.130000           4   

     holo_RMSD_min  holo_RMSD_max  holo

In [9]:
fi_df.to_csv('reviewed_RMSD_info.csv', index=False)

In [ ]:
!rm -rf /content/structures_from_pdb.zip
!zip -r structures_from_pdb.zip /content/cif_files/

  adding: content/cif_files/ (stored 0%)
  adding: content/cif_files/8d0p.pdb (deflated 75%)
  adding: content/cif_files/1dkg.pdb (deflated 76%)
  adding: content/cif_files/1us0.pdb (deflated 76%)
  adding: content/cif_files/3x39.pdb (deflated 76%)
  adding: content/cif_files/1gfs.pdb (deflated 76%)
  adding: content/cif_files/6v6u.pdb (deflated 75%)
  adding: content/cif_files/6fgd.pdb (deflated 75%)
  adding: content/cif_files/2rh2.pdb (deflated 77%)
  adding: content/cif_files/6iep.pdb (deflated 76%)
  adding: content/cif_files/3mjs.pdb (deflated 76%)
  adding: content/cif_files/5y1g.pdb (deflated 76%)
  adding: content/cif_files/3mjc.pdb (deflated 76%)
  adding: content/cif_files/7u8k.pdb (deflated 79%)
  adding: content/cif_files/2qw8.pdb (deflated 76%)
  adding: content/cif_files/6x2e.pdb (deflated 76%)
  adding: content/cif_files/1k6i.pdb (deflated 76%)
  adding: content/cif_files/1pdb.pdb (deflated 77%)
  adding: content/cif_files/8uw0.pdb (deflated 76%)
  adding: content/cif_f

In [10]:
!pip install gemmi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:00


# Skaičiuojame ligandų ir jonų kiekį holo struktūrose

In [11]:
import gemmi
from collections import Counter
import os
import pandas as pd
import re

# Define known ligands and ions
KNOWN_LIGANDS = {"BCB", "BCL", "CHL", "CLA", "CIT", "MYR", "OLA", "PLM", "HEC", "HEM", "NDP", "NAP", "NAD", "FAD", "GDP", "GTP", "AMP", "ATP", "ADP"}
KNOWN_LIGAND_MAP = {
    "BACTERIOCHLOROPHYLL B": "BCB",
    "BACTERIOCHLOROPHYLL": "BCL",
    "CHLOROPHYLL A": "CHL",
    "CHLOROPHYLL B": "CLA",
    "CITRIC ACID": "CIT",
    "MYRISTIC ACID": "MYR",
    "OLEIC ACID": "OLA",
    "PALMITIC ACID": "PLM",
    "HEMATOPOIETIC ENZYME COMPLEX": "HEC",
    "HEME C": "HEC",
    "HEME": "HEM",
    "PROTOPORPHYRIN IX CONTAINING FE": "HEM",
    "NADP NICOTINAMIDE-ADENINE-DINUCLEOTIDE PHOSPHATE": "NDP",
    "NADPH DIHYDRO-NICOTINAMIDE-ADENINE-DINUCLEOTIDE PHOSPHATE": "NDP",
    "NICOTINAMIDE-ADENINE-DINUCLEOTIDE": "NAD",
    "FLAVIN-ADENINE DINUCLEOTIDE": "FAD",
    "GUANOSINE-5'-DIPHOSPHATE": "GDP",
    "GUANOSINE-5'-TRIPHOSPHATE": "GTP",
    "ADENOSINE MONOPHOSPHATE": "AMP",
    "ADENOSINE-5'-TRIPHOSPHATE": "ATP",
    "ADENOSINE-5'-DIPHOSPHATE": "ADP",
}

KNOWN_IONS = {"MG", "ZN", "CL", "CA", "NA", "MN", "K", "FE", "CU", "CO"}
KNOWN_ION_MAP = {
    "MAGNESIUM ION": "MG",
    "ZINC ION": "ZN",
    "CHLORIDE ION": "CL",
    "CALCIUM ION": "CA",
    "SODIUM ION": "NA",
    "MANGANESE ION": "MN",
    "POTASSIUM ION": "K",
    "FE ION": "FE",
    "COPPER ION": "CU",
    "COBALT ION": "CO",
}

def parse_cif(file_path):
    doc = gemmi.cif.read_file(file_path)
    block = doc.sole_block()
    entity = []
    count = []
    for element in block.find_loop("_entity.pdbx_number_of_molecules"):
        count.append(element)
    for element in block.find_loop("_entity.pdbx_description"):
        entity.append(element)

    entity_dict = dict(zip(entity, count))
    filtered_ion_dict = dict()
    filtered_ligand_dict = dict()

    for key, value in entity_dict.items():
        key = key.strip("'")
        key = re.sub('\\(.*\\) ', '', key)
        key = key.strip("\"")

        if key in KNOWN_LIGAND_MAP:
            filtered_ligand_dict[KNOWN_LIGAND_MAP[key]] = value

        if key in KNOWN_ION_MAP:
            filtered_ion_dict[KNOWN_ION_MAP[key]] = value

    return filtered_ligand_dict, filtered_ion_dict

# Main function to process CIF files in a directory
def process_cif_directory(directory, holo_ids):
    results = []
    for file_name in os.listdir(directory):
        if file_name.endswith(".cif"):
            file_path = os.path.join(directory, file_name)

            if file_name[:4] not in holo_ids:
                continue

            print(f"Processing {file_name}...")
            filtered_ligand_dict, filtered_ion_dict  = parse_cif(file_path)

            results.append({
                "Holo_id": file_name[0:4],
                "Ligands": list(filtered_ligand_dict.keys()),
                "Ligand_count": list(filtered_ligand_dict.values()),
                "Ions": list(filtered_ion_dict.keys()),
                "Ions_count": list(filtered_ion_dict.values())
            })

    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    return results_df

df = pd.read_csv("reviewed_apo_holo_pair_ligand_and_ion_data.tsv", sep='\t')
holo_ids = list()

for id in df["Holo_PDB_ID"]:
    holo_ids.append(id)

# Path to directory containing CIF files
cif_directory = "/content/cif_files"

# Process CIF files and get results
cif_results_df = process_cif_directory(cif_directory, holo_ids)

cif_results_df.to_csv("ligand_and_ion_counts.csv", index=False)
print(cif_results_df)


Processing 7d4o.cif...
Processing 5yce.cif...
Processing 5gme.cif...
Processing 6x9a.cif...
Processing 3lq3.cif...
Processing 4cnz.cif...
Processing 6m65.cif...
Processing 4gkb.cif...
Processing 6tn5.cif...
Processing 3apt.cif...
Processing 1zk4.cif...
Processing 7r3d.cif...
Processing 4pf4.cif...
Processing 6jsc.cif...
Processing 2bcg.cif...
Processing 5aus.cif...
Processing 1m7h.cif...
Processing 8d0u.cif...
Processing 8dr9.cif...
Processing 2rk1.cif...
Processing 2w72.cif...
Processing 5c4n.cif...
Processing 1xgk.cif...
Processing 5dcy.cif...
Processing 3keq.cif...
Processing 5v1m.cif...
Processing 5j4l.cif...
Processing 4b1y.cif...
Processing 5iwu.cif...
Processing 1e7s.cif...
Processing 5b4t.cif...
Processing 6r3y.cif...
Processing 9b20.cif...
Processing 1kmv.cif...
Processing 2exv.cif...
Processing 5l3z.cif...
Processing 3g59.cif...
Processing 6dyf.cif...
Processing 6bmt.cif...
Processing 6ntb.cif...
Processing 4jp3.cif...
Processing 6nlg.cif...
Processing 6ok4.cif...
Processing 

# Suporuojame struktūras su uniprot identifikatoriumi

In [12]:
reviewed_holo_id_df = pd.read_csv("/content/reviewed_apo_holo_pair_ligand_and_ion_data.tsv", sep='\t')
uniprot_id_df = pd.read_csv("/content/apo_holo_pairs_with_RMSD_info.csv")

for index, row in uniprot_id_df.iterrows():
    uniprot_id_df.replace(row['holo_structure'], row['holo_structure'][0:4], inplace=True)
    uniprot_id_df.replace(row['uniprot_id'], re.sub('\\s.*', '', row['uniprot_id'].strip()), inplace=True)

print(uniprot_id_df['holo_structure'], reviewed_holo_id_df['Holo_PDB_ID'])

merged_df = uniprot_id_df.merge(
    reviewed_holo_id_df[['Holo_PDB_ID']],
    left_on='holo_structure',
    right_on='Holo_PDB_ID',
    how='inner'
)

reviewed_uniprot_ids = merged_df['uniprot_id'].tolist()

merged_df[['uniprot_id', 'Holo_PDB_ID']].to_csv('uniprot_holo_pdb_list.tsv', sep='\t', index=False)


0      2w72
1      6tn5
2      1x8q
3      1dgg
4      1e7s
       ... 
149    5j4l
150    8d0u
151    8evx
152    8u96
153    9b20
Name: holo_structure, Length: 154, dtype: object 0     2w72
1     6tn5
2     1x8q
3     1e7s
4     1xgk
      ... 
85    8crh
86    5j4l
87    8d0u
88    8u96
89    9b20
Name: Holo_PDB_ID, Length: 90, dtype: object


In [ ]:
!rm -rf /content/uniprot_entries
!rm -rf /content/uniprot_entries.zip

# Atsisiunčiame struktūrų sekas FASTA formatu naudojantis uniprot identifikatoriais ir API

In [13]:
import os
import requests

def download_fasta_sequences(uniprot_ids, output_dir):
    base_url = "https://rest.uniprot.org/uniprotkb/"
    headers = {"Accept": "text/x-fasta"}

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    for uniprot_id in uniprot_ids:
        uniprot_id = uniprot_id.strip()
        print(f"Fetching FASTA sequence for UniProt ID: {uniprot_id}...")
        url = f"{base_url}{uniprot_id}.fasta"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            # Save the FASTA sequence to a file named `uniprot_id.fasta`
            file_path = os.path.join(output_dir, f"{uniprot_id}.fasta")
            with open(file_path, "w") as fasta_file:
                fasta_file.write(response.text)
            print(f"Sequence saved to {file_path}")
        else:
            print(f"Failed to fetch FASTA sequence for {uniprot_id}. Status code: {response.status_code}")

# List of UniProt IDs to download
uniprot_holo_pdb_list = pd.read_csv("/content/uniprot_holo_pdb_list.tsv", sep='\t')

reviewed_uniprot_ids = uniprot_holo_pdb_list['uniprot_id'].tolist()
# Output directory
output_dir = "/content/uniprot_entries/"
os.makedirs(output_dir, exist_ok=True)
print(reviewed_uniprot_ids)
# Download the sequences
download_fasta_sequences(reviewed_uniprot_ids, output_dir)

['P68871', 'P07900', 'Q94734', 'P32055', 'Q5AU62', 'P00374', 'Q12657', 'P05091', 'P27616', 'P15121', 'P68135', 'P02185', 'P01123', 'Q84EX5', 'P0ABE7', 'O07347', 'O93715', 'Q53W82', 'Q60381', 'P00383', 'O28028', 'P56431', 'Q15GI4', 'P53355', 'Q13WK4', 'P0ABQ4', 'Q9Y259', 'Q6FNA9', 'P62826', 'Q8E565', 'P70731', 'Q93NW7', 'G3XD33', 'O28527', 'P54616', 'D0VWQ0', 'O15530', 'Q8TX37', 'C4LW95', 'Q9HZN8', 'P01116', 'Q9WYT0', 'A0A0H3KNE7', 'O66490', 'P37344', 'P9WGR1', 'O68098', 'G8NVB5', 'P15452', 'P0A9S1', 'O24146', 'P61586', 'Q92NR7', 'Q9HY79', 'K7WDL7', 'A0A0A1EQY0', 'A0A0H3AER7', 'Q97UL5', 'O32553', 'G0YYQ5', 'G9VYV4', 'Q8ZPX9', 'P05164', 'Q9BPX1', 'P9WP23', 'P50743', 'A0QUZ2', 'P00099', 'D6PBM7', 'Q8E3E8', 'Q9BQ65', 'P00720', 'Q03555', 'Q15286', 'Q8NTB9', 'B5II98', 'P9WFG7', 'A0A024B7W1', 'P0CE13', 'F7X6I3', 'P49366', 'P0DSP4', 'S9Q235', 'P0A6Y8', 'Q5SLG6', 'A0A0L0P1H8', 'O74036', 'Q9Y231', 'A0A0H3FR62', 'A0A0H3GVQ7']
Fetching FASTA sequence for UniProt ID: P68871...
Sequence saved to /co

In [ ]:
!zip -r uniprot_entries.zip /content/uniprot_entries/

  adding: content/uniprot_entries/ (stored 0%)
  adding: content/uniprot_entries/P0A6Y8.fasta (deflated 34%)
  adding: content/uniprot_entries/P01123.fasta (deflated 20%)
  adding: content/uniprot_entries/P0DSP4.fasta (deflated 27%)
  adding: content/uniprot_entries/A0A0L0P1H8.fasta (deflated 22%)
  adding: content/uniprot_entries/Q92NR7.fasta (deflated 24%)
  adding: content/uniprot_entries/P00099.fasta (deflated 18%)
  adding: content/uniprot_entries/O07347.fasta (deflated 32%)
  adding: content/uniprot_entries/P62826.fasta (deflated 22%)
  adding: content/uniprot_entries/P01116.fasta (deflated 23%)
  adding: content/uniprot_entries/A0A0A1EQY0.fasta (deflated 29%)
  adding: content/uniprot_entries/Q9WYT0.fasta (deflated 21%)
  adding: content/uniprot_entries/G3XD33.fasta (deflated 25%)
  adding: content/uniprot_entries/G9VYV4.fasta (deflated 28%)
  adding: content/uniprot_entries/O15530.fasta (deflated 31%)
  adding: content/uniprot_entries/P02185.fasta (deflated 16%)
  adding: conte

In [14]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.3 MB/s eta 0:00:00


# Sugeneruojame JSON darbų failus AlphaFold3 įrankiui

In [15]:
import os
import json
import pandas as pd
from Bio import SeqIO
import re
# Directory containing Uniprot FASTA files
fasta_dir = "uniprot_entries"

# Function to read the sequence from a Uniprot FASTA file
def get_sequence(uniprot_id):
    fasta_file = os.path.join(fasta_dir, f"{uniprot_id}.fasta")
    if not os.path.exists(fasta_file):
        print(f"FASTA file not found for Uniprot ID: {uniprot_id}")
        return None
    for record in SeqIO.parse(fasta_file, "fasta"):
        return str(record.seq)
    return None

# Function to create AlphaFold job JSON
def create_alphafold_job(job_name, sequence, ligands=None, ions=None):
    sequences = [
        {
            "proteinChain": {
                "sequence": sequence,
                "count": 1
            }
        }
    ]

    if ligands:
        for ligand, count in ligands.items():
            sequences.append({
                "ligand": {
                    "ligand": f"CCD_{ligand}",
                    "count": count
                }
            })

    if ions:
        for ion, count in ions.items():
            sequences.append({
                "ion": {
                    "ion": ion,
                    "count": count
                }
            })

    job_data = {
        "name": job_name,
        "modelSeeds": [],
        "sequences": sequences
    }

    return job_data

# Main script
if __name__ == "__main__":
    # Load data files
    input_tsv = "/content/reviewed_apo_holo_pair_ligand_and_ion_data.tsv"
    uniprot_holo_pdb_df = pd.read_csv("/content/uniprot_holo_pdb_list.tsv", sep='\t')
    ligand_ion_file = "/content/ligand_and_ion_counts.csv"
    output_file = "alphafold_jobs.json"

    # Read the main data file
    data = pd.read_csv(input_tsv, sep='\t')

    # Read ligand and ion counts
    counts_data = pd.read_csv(ligand_ion_file)
    ligand_counts = counts_data.set_index('Holo_id')['Ligand_count'].to_dict()
    ion_counts = counts_data.set_index('Holo_id')['Ions_count'].to_dict()
    all_jobs = []

    for _, row in data.iterrows():
        apo_id = row['Apo_PDB_ID']
        holo_id = row['Holo_PDB_ID']
        ligands = eval(row['Ligands']) if 'Ligands' in row and pd.notna(row['Ligands']) else []
        ions = eval(row['Ions']) if 'Ions' in row and pd.notna(row['Ions']) else []
        # Get Uniprot ID from mapping
        uniprot_id = uniprot_holo_pdb_df.loc[uniprot_holo_pdb_df['Holo_PDB_ID']== holo_id[0:4], 'uniprot_id'].iloc[0]
        print(uniprot_id, holo_id)
        if not uniprot_id:
            print(f"No Uniprot ID found for {holo_id}")
            continue

        # Get sequences for apo and holo structures
        apo_sequence = get_sequence(uniprot_id)
        holo_sequence = get_sequence(uniprot_id)

        if apo_sequence:
            apo_job = create_alphafold_job(job_name=f'{apo_id}_apo', sequence=apo_sequence)
            all_jobs.append(apo_job)
            print(f"Added AlphaFold job for apo: {apo_id}")

        if holo_sequence:
            ligand_dict = {}
            for i, ion in enumerate(ligands):
              counts = list(map(int, re.findall(r"\d+", ligand_counts.get(holo_id[0:4], 1))))
              ligand_dict[ion] = counts[i]

            ion_dict = {}
            for i, ion in enumerate(ions):
              counts = list(map(int, re.findall(r"\d+", ion_counts.get(holo_id[0:4], 1))))
              ion_dict[ion] = counts[i]
            holo_job = create_alphafold_job(job_name=f'{holo_id}_holo', sequence=holo_sequence, ligands=ligand_dict, ions=ion_dict)
            all_jobs.append(holo_job)
            print(f"Added AlphaFold job for holo: {holo_id}")
    print(len(all_jobs))

    def save_jobs_in_parts(all_jobs, output_file_base, max_jobs=100):
        total_jobs = len(all_jobs)
        num_parts = (total_jobs + max_jobs - 1) // max_jobs  # Ceiling division

        for i in range(num_parts):
            start = i * max_jobs
            end = min((i + 1) * max_jobs, total_jobs)
            part_jobs = all_jobs[start:end]

            filename = f"{output_file_base}_part_{i + 1}.json"
            with open(filename, "w") as outfile:
                json.dump(part_jobs, outfile, indent=4)
            print(f"Saved {len(part_jobs)} jobs to {filename}")

    output_file_base = "AlphaFold_jobs"
    save_jobs_in_parts(all_jobs, output_file_base, max_jobs=30)


P68871 2w72
Added AlphaFold job for apo: 5vmm
Added AlphaFold job for holo: 2w72
P07900 6tn5
Added AlphaFold job for apo: 5j80
Added AlphaFold job for holo: 6tn5
Q94734 1x8q
Added AlphaFold job for apo: 2ofm
Added AlphaFold job for holo: 1x8q
P32055 1e7s
Added AlphaFold job for apo: 1gfs
Added AlphaFold job for holo: 1e7s
Q5AU62 1xgk
Added AlphaFold job for apo: 1k6i
Added AlphaFold job for holo: 1xgk
P00374 1kmv
Added AlphaFold job for apo: 1pdb
Added AlphaFold job for holo: 1kmv
Q12657 1m7h
Added AlphaFold job for apo: 1d6j
Added AlphaFold job for holo: 1m7h
P05091 8dr9
Added AlphaFold job for apo: 3n80
Added AlphaFold job for holo: 8dr9
P27616 2cnq
Added AlphaFold job for apo: 2cnu
Added AlphaFold job for holo: 2cnq
P15121 1us0
Added AlphaFold job for apo: 1xgd
Added AlphaFold job for holo: 1us0
P68135 4b1y
Added AlphaFold job for apo: 7u8k
Added AlphaFold job for holo: 4b1y
P02185 5yce
Added AlphaFold job for apo: 1ufp
Added AlphaFold job for holo: 5yce
P01123 2bcg
Added AlphaFold 

# Išpakuojame prognozuotas struktūras

In [ ]:
!mkdir /content/all_folds
!unzip /content/folds_2025_04_16_12_36_part_1.zip -d /content/all_folds
!unzip /content/folds_2025_04_16_12_45_part_2.zip -d /content/all_folds
!unzip /content/folds_2025_04_17_08_00_part_3.zip -d /content/all_folds
!unzip /content/folds_2025_04_17_07_46_part_4.zip -d /content/all_folds
!unzip /content/folds_2025_04_17_08_13_part_5.zip -d /content/all_folds
!unzip /content/folds_2025_04_17_09_40_part_6.zip -d /content/all_folds

Archive:  /content/folds_2025_04_16_12_36_part_1.zip
  inflating: /content/all_folds/terms_of_use.md  
  inflating: /content/all_folds/3n80_apo/fold_3n80_apo_job_request.json  
  inflating: /content/all_folds/3n80_apo/fold_3n80_apo_model_0.cif  
  inflating: /content/all_folds/3n80_apo/fold_3n80_apo_summary_confidences_0.json  
  inflating: /content/all_folds/3n80_apo/fold_3n80_apo_full_data_0.json  
  inflating: /content/all_folds/3n80_apo/fold_3n80_apo_model_1.cif  
  inflating: /content/all_folds/3n80_apo/fold_3n80_apo_summary_confidences_1.json  
  inflating: /content/all_folds/3n80_apo/fold_3n80_apo_full_data_1.json  
  inflating: /content/all_folds/3n80_apo/fold_3n80_apo_model_2.cif  
  inflating: /content/all_folds/3n80_apo/fold_3n80_apo_summary_confidences_2.json  
  inflating: /content/all_folds/3n80_apo/fold_3n80_apo_full_data_2.json  
  inflating: /content/all_folds/3n80_apo/fold_3n80_apo_model_3.cif  
  inflating: /content/all_folds/3n80_apo/fold_3n80_apo_summary_confidence

# Skaičiuojame plDDt įverčius

In [ ]:
import os
import json
import re

# Directory containing your JSON files
data_dir = "all_folds"

# Output TSV file
output_file = "plddt_results.tsv"

# 1. Discover all fold numbers across all folders
fold_numbers = set()
pattern = re.compile(r".*full_data_(\d+)\.json$")
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    if not os.path.isdir(folder_path):
        continue
    for filename in os.listdir(folder_path):
        m = pattern.match(filename)
        print(filename, m)
        if m:
            fold_numbers.add(int(m.group(1)))

# Sort so columns come out in numeric order
fold_numbers = sorted(fold_numbers)

# 2. Write header
with open(output_file, "w") as outfile:
    header = ["PDB_NAME"] + [f"PLDDT_{n}" for n in fold_numbers]
    outfile.write("\t".join(header) + "\n")

    # 3. Process each folder and collect averages
    for folder in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder)
        if not os.path.isdir(folder_path):
            continue

        # compute avg pLDDT per fold in this folder
        avg_by_fold = {}
        for filename in os.listdir(folder_path):
            m = pattern.match(filename)
            if not m:
                continue
            fold_idx = int(m.group(1))
            json_path = os.path.join(folder_path, filename)
            with open(json_path, "r") as f:
                data = json.load(f)
            atom_plddts = data.get("atom_plddts", [])
            if atom_plddts:
                avg_plddt = sum(atom_plddts) / len(atom_plddts)
                avg_by_fold[fold_idx] = avg_plddt

        # build row: PDB name (uppercase) + values or blanks
        row = [folder.upper()]
        for n in fold_numbers:
            if n in avg_by_fold:
                row.append(f"{avg_by_fold[n]:.2f}")
            else:
                row.append("")  # or use "NA" if you prefer
        outfile.write("\t".join(row) + "\n")

print(f"pLDDT calculations completed. Results saved in {output_file}.")


fold_5jbj_apo_full_data_0.json <re.Match object; span=(0, 30), match='fold_5jbj_apo_full_data_0.json'>
fold_5jbj_apo_model_1.cif None
fold_5jbj_apo_summary_confidences_0.json None
fold_5jbj_apo_full_data_2.json <re.Match object; span=(0, 30), match='fold_5jbj_apo_full_data_2.json'>
fold_5jbj_apo_full_data_1.json <re.Match object; span=(0, 30), match='fold_5jbj_apo_full_data_1.json'>
fold_5jbj_apo_full_data_3.json <re.Match object; span=(0, 30), match='fold_5jbj_apo_full_data_3.json'>
fold_5jbj_apo_full_data_4.json <re.Match object; span=(0, 30), match='fold_5jbj_apo_full_data_4.json'>
fold_5jbj_apo_model_4.cif None
fold_5jbj_apo_model_2.cif None
fold_5jbj_apo_job_request.json None
fold_5jbj_apo_model_0.cif None
fold_5jbj_apo_model_3.cif None
fold_5jbj_apo_summary_confidences_2.json None
fold_5jbj_apo_summary_confidences_1.json None
fold_5jbj_apo_summary_confidences_4.json None
fold_5jbj_apo_summary_confidences_3.json None
fold_5iqx_holo_model_4.cif None
fold_5iqx_holo_model_0.cif None


# Skaičiuojame RMSD įverčius

In [ ]:
from IPython.utils import io
import tqdm.notebook
import os
"""The PyMOL installation is done inside two nested context managers. This approach
was inspired by Dr. Christopher Schlick's (of the Phenix group at
Lawrence Berkeley National Laboratory) method for installing cctbx
in a Colab Notebook. He presented his work on September 1, 2021 at the IUCr
Crystallographic Computing School. I adapted Chris's approach here. It replaces my first approach
that requires seven steps. My approach was presentated at the SciPy2021 conference
in July 2021 and published in the
[proceedings](http://conference.scipy.org/proceedings/scipy2021/blaine_mooers.html).
The new approach is easier for beginners to use. The old approach is easier to debug
and could be used as a back-up approach.

Thank you to Professor David Oppenheimer of the University of Florida for suggesting the use mamba and of Open Source PyMOL.
"""
total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:

        !pip install -q condacolab
        import condacolab
        condacolab.install()
        pbar.update(10)

        import sys
        sys.path.append('/usr/local/lib/python3.10/site-packages/')
        pbar.update(20)

        # Install PyMOL
        %shell mamba install pymol-open-source --yes

        pbar.update(100)


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
!unzip /content/structures_from_pdb.zip -d /content/cif_files

Archive:  /content/structures_from_pdb.zip
   creating: /content/cif_files/content/cif_files/
  inflating: /content/cif_files/content/cif_files/8uw0.cif  
  inflating: /content/cif_files/content/cif_files/6ekk.cif  
  inflating: /content/cif_files/content/cif_files/6qaj.cif  
  inflating: /content/cif_files/content/cif_files/6nlf.cif  
  inflating: /content/cif_files/content/cif_files/4qnr.cif  
  inflating: /content/cif_files/content/cif_files/3keq.cif  
  inflating: /content/cif_files/content/cif_files/3g59.cif  
  inflating: /content/cif_files/content/cif_files/2cd9.cif  
  inflating: /content/cif_files/content/cif_files/8onv.cif  
  inflating: /content/cif_files/content/cif_files/6zx9.cif  
  inflating: /content/cif_files/content/cif_files/6m65.cif  
  inflating: /content/cif_files/content/cif_files/2qw8.cif  
  inflating: /content/cif_files/content/cif_files/4bga.cif  
  inflating: /content/cif_files/content/cif_files/6jsd.cif  
  inflating: /content/cif_files/content/cif_files/5j

In [ ]:
from pymol import cmd
# from IPython.display import Image
import pandas as pd
df = pd.read_csv("/content/reviewed_apo_holo_pair_ligand_and_ion_data.tsv", sep='\t')
pairs = []
for index, row in df.iterrows():
        apo = row['Apo_PDB_ID']
        holo = row['Holo_PDB_ID']
        pairs.append([apo, holo])
RMSD_map = dict()
for pair in pairs:
  cmd.reinitialize()
  cmd.load(f"/content/cif_files/content/cif_files/{pair[0][0:4]}.cif", "str1")
  cmd.load(f"/content/cif_files/content/cif_files/{pair[1][0:4]}.cif", "str2")
  RMSD = cmd.align("str2", "str1")[0]  # Align structure2 to structure1
  RMSD_map[f'{pair[0][0:4]}_{pair[1][0:4]}'] = RMSD
  print(f"RMSD between {pair[0]} and {pair[1]}: {RMSD}")

  ->./XE.cif
RMSD between 5vmm and 2w72: 3.785067081451416
RMSD between 5j80 and 6tn5: 0.2576241195201874
RMSD between 2ofm and 1x8q: 0.2636294662952423
RMSD between 1gfs and 1e7s: 0.2641831338405609
RMSD between 1k6i and 1xgk: 0.1773521453142166
RMSD between 1pdb and 1kmv: 0.6890032887458801
RMSD between 1d6j and 1m7h: 0.8921723961830139
RMSD between 3n80 and 8dr9: 51.66289138793945
RMSD between 2cnu and 2cnq: 0.9901812672615051
RMSD between 1xgd and 1us0: 0.5888625979423523
RMSD between 7u8k and 4b1y: 1.5697057247161865
RMSD between 1ufp and 5yce: 0.6060529351234436
RMSD between 3cue and 2bcg: 31.333511352539062
RMSD between 7a2b and 1zk4: 0.1095983237028122
RMSD between 1apc and 6dyf: 3.8038365840911865
RMSD between 1ffh and 2j46: 0.45971956849098206
  ->./ZN.cif
RMSD between 2cd9 and 2cdb: 0.29073020815849304
RMSD between 4jp2 and 4jp3: 0.1582803726196289
RMSD between 2j9d and 2j9e: 0.25424060225486755
RMSD between 2rh2 and 2rk1: 0.08633699268102646
RMSD between 2g9i and 7uld: 0.44

In [ ]:
df['APO_vs_HOLO'] = RMSD_map.values()
print(df)

   PDB_ID   Status  Conformers  Ligands    Ions Apo_PDB_ID Holo_PDB_ID  \
0    2w72  Success           3  ['HEM']   ['K']       5vmm        2w72   
1    6tn5  Success           1       []  ['CL']       5j80        6tn5   
2    1x8q  Success           1  ['HEM']      []       2ofm        1x8q   
3    1e7s  Success           1  ['NAP']      []       1gfs        1e7s   
4    1xgk  Success           1       []   ['K']       1k6i        1xgk   
..    ...      ...         ...      ...     ...        ...         ...   
85   8crh  Success           1  ['NDP']      []       7zzx        8crh   
86   5j4l  Success           1       []  ['CL']       6tuu        5j4l   
87   8d0u  Success           1  ['GDP']      []       8d0p        8d0u   
88   8u96  Success           1  ['ATP']  ['CL']       8sbn        8u96   
89   9b20  Success           1       []  ['MG']       9b1z        9b20   

    APO_vs_HOLO  
0      3.785067  
1      0.257624  
2      0.263629  
3      0.264183  
4      0.177352  
.. 

In [ ]:
# APO X HOLO_ALPHA
new_pairs = []
for pair in pairs:
  alpha_model = pair[1] + '_holo'
  # print(alpha_model)
  new_pairs.append([pair[0], alpha_model])
RMSD_map = dict()
pairs_w_model = dict()
model_w_pdb_ids = pd.DataFrame(columns=['apo_pdb_id', 'holo_pdb_id', 'alphafold_model_id'])
model = 0
for pair in new_pairs:
  RMSD = 0
  for i in range(0, 5):
    cmd.reinitialize()
    cmd.load(f"/content/cif_files/content/cif_files/{pair[0][0:4]}.cif", "str1")
    cmd.load(f"/content/all_folds/{pair[1]}/fold_{pair[1]}_model_{i}.cif", "str2")
    RMSD_new = cmd.align("str2", "str1")[0]  # Align structure2 to structure1
    if RMSD < RMSD_new:
      RMSD = RMSD_new
      model = i
  RMSD_map[f'{pair[0][0:4]}_{pair[1][0:4]}'] = RMSD
  new_row = {
    'apo_pdb_id':          pair[0],
    'holo_pdb_id':         pair[1][:4],
    'alphafold_model_id':  model
  }
  model_w_pdb_ids.loc[len(model_w_pdb_ids)] = new_row
#   model_w_pdb_ids.append({'apo_pdb_id': pair[0], 'holo_pdb_id': pair[1][:4], 'alphafold_model_id': model}, ignore_index=True)
  pairs_w_model[pair[0]] = model
  print(f"RMSD between {pair[0]} and {pair[1][0:4]}_alpha: {RMSD}")

RMSD between 5vmm and 2w72_alpha: 0.7270038723945618
RMSD between 5j80 and 6tn5_alpha: 0.27227675914764404
RMSD between 2ofm and 1x8q_alpha: 0.3280476927757263
RMSD between 1gfs and 1e7s_alpha: 0.28335633873939514
RMSD between 1k6i and 1xgk_alpha: 0.24348580837249756
RMSD between 1pdb and 1kmv_alpha: 0.6770316362380981
RMSD between 1d6j and 1m7h_alpha: 1.0385462045669556
RMSD between 3n80 and 8dr9_alpha: 0.17998363077640533
RMSD between 2cnu and 2cnq_alpha: 0.5191517472267151
RMSD between 1xgd and 1us0_alpha: 0.5088538527488708
RMSD between 7u8k and 4b1y_alpha: 1.3108265399932861
RMSD between 1ufp and 5yce_alpha: 0.4627609848976135
RMSD between 3cue and 2bcg_alpha: 1.9346128702163696
RMSD between 7a2b and 1zk4_alpha: 0.171620175242424
RMSD between 1apc and 6dyf_alpha: 3.3999927043914795
RMSD between 1ffh and 2j46_alpha: 0.6109067797660828
RMSD between 2cd9 and 2cdb_alpha: 0.4408150315284729
RMSD between 4jp2 and 4jp3_alpha: 0.31142765283584595
RMSD between 2j9d and 2j9e_alpha: 0.303088

In [ ]:
df['APO_vs_HOLO_ALPHA'] = RMSD_map.values()
print(df)
print(model_w_pdb_ids)
model_w_pdb_ids.to_csv('model_w_pdb_ids.csv', index=False)

   PDB_ID   Status  Conformers  Ligands    Ions Apo_PDB_ID Holo_PDB_ID  \
0    2w72  Success           3  ['HEM']   ['K']       5vmm        2w72   
1    6tn5  Success           1       []  ['CL']       5j80        6tn5   
2    1x8q  Success           1  ['HEM']      []       2ofm        1x8q   
3    1e7s  Success           1  ['NAP']      []       1gfs        1e7s   
4    1xgk  Success           1       []   ['K']       1k6i        1xgk   
..    ...      ...         ...      ...     ...        ...         ...   
85   8crh  Success           1  ['NDP']      []       7zzx        8crh   
86   5j4l  Success           1       []  ['CL']       6tuu        5j4l   
87   8d0u  Success           1  ['GDP']      []       8d0p        8d0u   
88   8u96  Success           1  ['ATP']  ['CL']       8sbn        8u96   
89   9b20  Success           1       []  ['MG']       9b1z        9b20   

    APO_vs_HOLO  APO_vs_HOLO_ALPHA  
0      3.785067           0.727004  
1      0.257624           0.272277  


In [ ]:
# HOLO X APO_ALPHA
new_pairs = []
for pair in pairs:
  alpha_model = pair[0] + '_apo'
  # print(alpha_model)
  new_pairs.append([pair[1], alpha_model, pairs_w_model[pair[0]]])
RMSD_map = dict()
for pair in new_pairs:
  # RMSD = 100
  cmd.reinitialize()
  cmd.load(f"/content/cif_files/content/cif_files/{pair[0]}.cif", "str1")
  cmd.load(f"/content/all_folds/{pair[1]}/fold_{pair[1]}_model_{pair[2]}.cif", "str2")
  RMSD = cmd.align("str2", "str1")[0]  # Align structure2 to structure1

      # print(f"Model: {i}")
  RMSD_map[f'{pair[0]}_{pair[1][0:4]}'] = RMSD
  print(f"RMSD between {pair[0]} and {pair[1][0:4]}_alpha: {RMSD}")

RMSD between 2w72 and 5vmm_alpha: 0.6015127301216125
RMSD between 6tn5 and 5j80_alpha: 0.44321155548095703
RMSD between 1x8q and 2ofm_alpha: 0.2589937150478363
RMSD between 1e7s and 1gfs_alpha: 0.25430214405059814
RMSD between 1xgk and 1k6i_alpha: 0.17159682512283325
RMSD between 1kmv and 1pdb_alpha: 0.5687075853347778
RMSD between 1m7h and 1d6j_alpha: 0.564735472202301
RMSD between 8dr9 and 3n80_alpha: 0.22595389187335968
RMSD between 2cnq and 2cnu_alpha: 0.7983500957489014
RMSD between 1us0 and 1xgd_alpha: 0.2003105729818344
RMSD between 4b1y and 7u8k_alpha: 1.455474853515625
RMSD between 5yce and 1ufp_alpha: 0.4989243149757385
RMSD between 2bcg and 3cue_alpha: 0.526677131652832
RMSD between 1zk4 and 7a2b_alpha: 0.17777231335639954
RMSD between 6dyf and 1apc_alpha: 0.3659241795539856
RMSD between 2j46 and 1ffh_alpha: 1.510663628578186
RMSD between 2cdb and 2cd9_alpha: 0.17002923786640167
RMSD between 4jp3 and 4jp2_alpha: 0.3752644956111908
RMSD between 2j9e and 2j9d_alpha: 0.36695444

In [ ]:
df = pd.read_csv("/content/RMSD_results.csv", sep=',')
df['HOLO_vs_APO_ALPHA'] = RMSD_map.values()
print(df)
df.to_csv('RMSD_results.csv', index=False)

   PDB_ID   Status  Conformers  Ligands    Ions Apo_PDB_ID Holo_PDB_ID  \
0    2w72  Success           3  ['HEM']   ['K']       5vmm        2w72   
1    6tn5  Success           1       []  ['CL']       5j80        6tn5   
2    1x8q  Success           1  ['HEM']      []       2ofm        1x8q   
3    1e7s  Success           1  ['NAP']      []       1gfs        1e7s   
4    1xgk  Success           1       []   ['K']       1k6i        1xgk   
..    ...      ...         ...      ...     ...        ...         ...   
85   8crh  Success           1  ['NDP']      []       7zzx        8crh   
86   5j4l  Success           1       []  ['CL']       6tuu        5j4l   
87   8d0u  Success           1  ['GDP']      []       8d0p        8d0u   
88   8u96  Success           1  ['ATP']  ['CL']       8sbn        8u96   
89   9b20  Success           1       []  ['MG']       9b1z        9b20   

    APO_vs_HOLO  APO_vs_HOLO_ALPHA  APO_vs_APO_ALPHA  HOLO_vs_HOLO_ALPHA  \
0      3.785067           0.727004 

In [ ]:
# APO X APO_ALPHA
new_pairs = []
for pair in pairs:
  alpha_model = pair[0] + '_apo'
  # print(alpha_model)
  new_pairs.append([pair[0], alpha_model, pairs_w_model[pair[0]]])
RMSD_map = dict()
for pair in new_pairs:
  # RMSD = 100
  cmd.reinitialize()
  cmd.load(f"/content/cif_files/content/cif_files/{pair[0]}.cif", "str1")
  cmd.load(f"/content/all_folds/{pair[1]}/fold_{pair[1]}_model_{pair[2]}.cif", "str2")
  RMSD = cmd.align("str2", "str1")[0]  # Align structure2 to structure1

      # print(f"Model: {i}")
  RMSD_map[f'{pair[0]}_{pair[1][0:4]}'] = RMSD
  print(f"RMSD between {pair[0]} and {pair[1][0:4]}_alpha: {RMSD}")

RMSD between 5vmm and 5vmm_alpha: 0.6351919174194336
RMSD between 5j80 and 5j80_alpha: 0.4210648238658905
RMSD between 2ofm and 2ofm_alpha: 0.3524399995803833
RMSD between 1gfs and 1gfs_alpha: 0.24799276888370514
RMSD between 1k6i and 1k6i_alpha: 0.22991079092025757
RMSD between 1pdb and 1pdb_alpha: 0.6412376165390015
RMSD between 1d6j and 1d6j_alpha: 0.8791496753692627
RMSD between 3n80 and 3n80_alpha: 0.17029742896556854
RMSD between 2cnu and 2cnu_alpha: 0.8026918768882751
RMSD between 1xgd and 1xgd_alpha: 0.46913692355155945
RMSD between 7u8k and 7u8k_alpha: 2.1467695236206055
RMSD between 1ufp and 1ufp_alpha: 0.31079816818237305
RMSD between 3cue and 3cue_alpha: 1.8968489170074463
RMSD between 7a2b and 7a2b_alpha: 0.16601382195949554
RMSD between 1apc and 1apc_alpha: 3.3575353622436523
RMSD between 1ffh and 1ffh_alpha: 1.095964789390564
RMSD between 2cd9 and 2cd9_alpha: 0.3199032247066498
RMSD between 4jp2 and 4jp2_alpha: 0.3846393823623657
RMSD between 2j9d and 2j9d_alpha: 0.34492

In [ ]:
df['APO_vs_APO_ALPHA'] = RMSD_map.values()
print(df)

   PDB_ID   Status  Conformers  Ligands    Ions Apo_PDB_ID Holo_PDB_ID  \
0    2w72  Success           3  ['HEM']   ['K']       5vmm        2w72   
1    6tn5  Success           1       []  ['CL']       5j80        6tn5   
2    1x8q  Success           1  ['HEM']      []       2ofm        1x8q   
3    1e7s  Success           1  ['NAP']      []       1gfs        1e7s   
4    1xgk  Success           1       []   ['K']       1k6i        1xgk   
..    ...      ...         ...      ...     ...        ...         ...   
85   8crh  Success           1  ['NDP']      []       7zzx        8crh   
86   5j4l  Success           1       []  ['CL']       6tuu        5j4l   
87   8d0u  Success           1  ['GDP']      []       8d0p        8d0u   
88   8u96  Success           1  ['ATP']  ['CL']       8sbn        8u96   
89   9b20  Success           1       []  ['MG']       9b1z        9b20   

    APO_vs_HOLO  APO_vs_HOLO_ALPHA  APO_vs_APO_ALPHA  
0      3.785067           0.727004          0.635192  
1

In [ ]:
# HOLO X HOLO_ALPHA
new_pairs = []
for pair in pairs:
  alpha_model = pair[1] + '_holo'
  # print(alpha_model)
  new_pairs.append([pair[1], alpha_model, pairs_w_model[pair[0]]])
RMSD_map = dict()
for pair in new_pairs:
  # RMSD = 100
  cmd.reinitialize()
  cmd.load(f"/content/cif_files/content/cif_files/{pair[0]}.cif", "str1")
  cmd.load(f"/content/all_folds/{pair[1]}/fold_{pair[1]}_model_{pair[2]}.cif", "str2")
  RMSD = cmd.align("str2", "str1")[0]  # Align structure2 to structure1

      # print(f"Model: {i}")
  RMSD_map[f'{pair[0]}_{pair[1][0:4]}'] = RMSD
  print(f"RMSD between {pair[0]} and {pair[1][0:4]}_alpha: {RMSD}")

RMSD between 2w72 and 2w72_alpha: 0.7123081684112549
RMSD between 6tn5 and 6tn5_alpha: 0.26785922050476074
RMSD between 1x8q and 1x8q_alpha: 0.2316313534975052
RMSD between 1e7s and 1e7s_alpha: 0.20285482704639435
RMSD between 1xgk and 1xgk_alpha: 0.19433462619781494
RMSD between 1kmv and 1kmv_alpha: 0.5250555872917175
RMSD between 1m7h and 1m7h_alpha: 0.5978659987449646
RMSD between 8dr9 and 8dr9_alpha: 0.20531490445137024
RMSD between 2cnq and 2cnq_alpha: 1.078899621963501
RMSD between 1us0 and 1us0_alpha: 0.13766351342201233
RMSD between 4b1y and 4b1y_alpha: 0.4797632694244385
RMSD between 5yce and 5yce_alpha: 0.3338206708431244
RMSD between 2bcg and 2bcg_alpha: 0.255928635597229
RMSD between 1zk4 and 1zk4_alpha: 0.12602490186691284
RMSD between 6dyf and 6dyf_alpha: 0.3284011483192444
RMSD between 2j46 and 2j46_alpha: 0.7227142453193665
RMSD between 2cdb and 2cdb_alpha: 0.27384236454963684
RMSD between 4jp3 and 4jp3_alpha: 0.32368558645248413
RMSD between 2j9e and 2j9e_alpha: 0.2482

In [ ]:
df['HOLO_vs_HOLO_ALPHA'] = RMSD_map.values()
print(df)

   PDB_ID   Status  Conformers  Ligands    Ions Apo_PDB_ID Holo_PDB_ID  \
0    2w72  Success           3  ['HEM']   ['K']       5vmm        2w72   
1    6tn5  Success           1       []  ['CL']       5j80        6tn5   
2    1x8q  Success           1  ['HEM']      []       2ofm        1x8q   
3    1e7s  Success           1  ['NAP']      []       1gfs        1e7s   
4    1xgk  Success           1       []   ['K']       1k6i        1xgk   
..    ...      ...         ...      ...     ...        ...         ...   
85   8crh  Success           1  ['NDP']      []       7zzx        8crh   
86   5j4l  Success           1       []  ['CL']       6tuu        5j4l   
87   8d0u  Success           1  ['GDP']      []       8d0p        8d0u   
88   8u96  Success           1  ['ATP']  ['CL']       8sbn        8u96   
89   9b20  Success           1       []  ['MG']       9b1z        9b20   

    APO_vs_HOLO  APO_vs_HOLO_ALPHA  APO_vs_APO_ALPHA  HOLO_vs_HOLO_ALPHA  
0      3.785067           0.727004  

In [ ]:
# APO_ALPHA X HOLO_ALPHA
new_pairs = []
for pair in pairs:
  alpha_model_apo = pair[0] + '_apo'
  alpha_model_holo = pair[1] + '_holo'
  # print(alpha_model)
  new_pairs.append([alpha_model_apo, alpha_model_holo, pairs_w_model[pair[0]]])
RMSD_map = dict()
for pair in new_pairs:
  # RMSD = 100
  cmd.reinitialize()
  cmd.load(f"/content/all_folds/{pair[0]}/fold_{pair[0]}_model_{pair[2]}.cif", "str1")
  cmd.load(f"/content/all_folds/{pair[1]}/fold_{pair[1]}_model_{pair[2]}.cif", "str2")
  RMSD = cmd.align("str2", "str1")[0]  # Align structure2 to structure1

      # print(f"Model: {i}")
  RMSD_map[f'{pair[0][0:4]}_{pair[1][0:4]}'] = RMSD
  print(f"RMSD between {pair[0][0:4]}_apo_alpha and {pair[1][0:4]}_holo_alpha: {RMSD}")

RMSD between 5vmm_apo_alpha and 2w72_holo_alpha: 0.3569370210170746
RMSD between 5j80_apo_alpha and 6tn5_holo_alpha: 4.640476226806641
RMSD between 2ofm_apo_alpha and 1x8q_holo_alpha: 0.1834811568260193
RMSD between 1gfs_apo_alpha and 1e7s_holo_alpha: 0.14423111081123352
RMSD between 1k6i_apo_alpha and 1xgk_holo_alpha: 0.15292668342590332
RMSD between 1pdb_apo_alpha and 1kmv_holo_alpha: 0.1663416177034378
RMSD between 1d6j_apo_alpha and 1m7h_holo_alpha: 0.341519832611084
RMSD between 3n80_apo_alpha and 8dr9_holo_alpha: 0.137351393699646
RMSD between 2cnu_apo_alpha and 2cnq_holo_alpha: 0.7092178463935852
RMSD between 1xgd_apo_alpha and 1us0_holo_alpha: 0.1540887951850891
RMSD between 7u8k_apo_alpha and 4b1y_holo_alpha: 1.2708673477172852
RMSD between 1ufp_apo_alpha and 5yce_holo_alpha: 0.32955530285835266
RMSD between 3cue_apo_alpha and 2bcg_holo_alpha: 0.3593992590904236
RMSD between 7a2b_apo_alpha and 1zk4_holo_alpha: 0.15144166350364685
RMSD between 1apc_apo_alpha and 6dyf_holo_alpha

In [ ]:
df['APO_ALPHA_vs_HOLO_ALPHA'] = RMSD_map.values()
print(df)

   PDB_ID   Status  Conformers  Ligands    Ions Apo_PDB_ID Holo_PDB_ID  \
0    2w72  Success           3  ['HEM']   ['K']       5vmm        2w72   
1    6tn5  Success           1       []  ['CL']       5j80        6tn5   
2    1x8q  Success           1  ['HEM']      []       2ofm        1x8q   
3    1e7s  Success           1  ['NAP']      []       1gfs        1e7s   
4    1xgk  Success           1       []   ['K']       1k6i        1xgk   
..    ...      ...         ...      ...     ...        ...         ...   
85   8crh  Success           1  ['NDP']      []       7zzx        8crh   
86   5j4l  Success           1       []  ['CL']       6tuu        5j4l   
87   8d0u  Success           1  ['GDP']      []       8d0p        8d0u   
88   8u96  Success           1  ['ATP']  ['CL']       8sbn        8u96   
89   9b20  Success           1       []  ['MG']       9b1z        9b20   

    APO_vs_HOLO  APO_vs_HOLO_ALPHA  APO_vs_APO_ALPHA  HOLO_vs_HOLO_ALPHA  \
0      3.785067           0.727004 

In [ ]:
df.to_csv('RMSD_results.csv', index=False)